In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA

In [3]:
data=pd.read_excel("EastWestAirlines.xlsx")
data.head()

,East-West Airlines is trying to learn more about its customers. Key issues are their,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"flying patterns, earning and use of frequent f...",NaN,NaN,NaN,NaN
1,card. The task is to identify customer segmen...,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,Source: Based upon real business data; company...,NaN,NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 5 columns):
 #   Column                                                                                 Non-Null Count  Dtype 
---  ------                                                                                 --------------  ----- 
 0   East-West Airlines is trying to learn more about its customers.  Key issues are their  18 non-null     object
 1   Unnamed: 1                                                                             13 non-null     object
 2   Unnamed: 2                                                                             6 non-null      object
 3   Unnamed: 3                                                                             13 non-null     object
 4   Unnamed: 4                                                                             18 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [5]:
data.isnull().sum()

East-West Airlines is trying to learn more about its customers.  Key issues are their     8
Unnamed: 1                                                                               13
Unnamed: 2                                                                               20
Unnamed: 3                                                                               13
Unnamed: 4                                                                                8
dtype: int64

In [6]:
data.describe()

,East-West Airlines is trying to learn more about its customers. Key issues are their,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,18,13,6,13,18
unique,18,3,3,3,18
top,"flying patterns, earning and use of frequent f...",NUMBER,1,Raw,Description
freq,1,9,3,9,1


In [7]:
data=data.drop_duplicates()
data

,East-West Airlines is trying to learn more about its customers. Key issues are their,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"flying patterns, earning and use of frequent f...",NaN,NaN,NaN,NaN
1,card. The task is to identify customer segmen...,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
4,Source: Based upon real business data; company...,NaN,NaN,NaN,NaN
5,(c) 2016 Galit Shmueli and Peter Bruce,NaN,NaN,NaN,NaN
8,Field Name,Data Type,Max Data Length,Raw Data or Telcom Created Field?,Description
9,ID#,NUMBER,NaN,Telcom,Unique ID
10,Balance,NUMBER,8,Raw,Number of miles eligible for award travel
11,Qual_miles,NUMBER,8,Raw,Number of miles counted as qualifying for Topf...
12,cc1_miles,CHAR,1,Raw,Number of miles earned with freq. flyer credit...


##### DATA PREPROCESSING

In [8]:
data.columns

Index(['East-West Airlines is trying to learn more about its customers.  Key issues are their',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],
      dtype='object')

In [9]:



print(data.head())
# Check if the DataFrame has any columns first
if not data.empty:
    # Get numeric columns
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    print("Numeric columns:", num_cols)
    
    # Only try to describe if there are numeric columns
    if len(num_cols) > 0:
        print(data[num_cols].describe().transpose())
    else:
        print("No numeric columns found in the DataFrame")
else:
    print("DataFrame is empty")

  East-West Airlines is trying to learn more about its customers.  Key issues are their  \
0  flying patterns, earning and use of frequent f...                                      
1  card.  The task is to identify customer segmen...                                      
2                                                NaN                                      
4  Source: Based upon real business data; company...                                      
5             (c) 2016 Galit Shmueli and Peter Bruce                                      

  Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  
0        NaN        NaN        NaN        NaN  
1        NaN        NaN        NaN        NaN  
2        NaN        NaN        NaN        NaN  
4        NaN        NaN        NaN        NaN  
5        NaN        NaN        NaN        NaN  
Numeric columns: []
No numeric columns found in the DataFrame


In [15]:
# Suggestion: plot histograms for numeric cols (univariate)
for col in num_cols:
    plt.figure(figsize=(6,4))
    plt.hist(data[col].dropna(), bins=30,color='skyblue',edgecolor='black')
    plt.title(f"Histogram: {col}")
    plt.xlabel(col)
    plt.ylabel("count")
plt.show()


In [16]:
import matplotlib.pyplot as plt
import math

n = len(num_cols)
cols = 3  # number of plots per row
rows = math.ceil(n / cols)

plt.figure(figsize=(15, rows * 4))
for i, col in enumerate(num_cols, 1):
    plt.subplot(rows, cols, i)
    plt.hist(data[col].dropna(), bins=30, color='lightgreen', edgecolor='black')
    plt.title(col)
plt.tight_layout()
plt.show()


<Figure size 1500x0 with 0 Axes>

In [11]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 12))
for i, col in enumerate(num_cols):
    plt.subplot(3, 4, i + 1)
    plt.hist(data[col].dropna(), bins=30)
    plt.title(col)
plt.tight_layout()
plt.show()


<Figure size 1500x1200 with 0 Axes>

In [18]:
import math
%matplotlib inline
num_cols = data.select_dtypes(include=['int64', 'float64']).columns
rows = math.ceil(len(num_cols)/3)

plt.figure(figsize=(15, 4*rows))
for i, col in enumerate(num_cols, 1):
    plt.subplot(rows, 3, i)
    plt.hist(data[col].dropna(), bins=30, color='lightgreen', edgecolor='black')
    plt.title(col)
plt.tight_layout()
plt.show()


<Figure size 1500x0 with 0 Axes>

In [19]:
# --- imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- magic command for inline plotting ---
%matplotlib inline

# --- now your plotting code ---
num_cols = data.select_dtypes(include=['int64', 'float64']).columns

for col in num_cols:
    plt.figure(figsize=(6,4))
    plt.hist(data[col].dropna(), bins=30, color='skyblue', edgecolor='black')
    plt.title(f"Histogram: {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.grid(True)
    plt.show()


In [14]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Create or load your dataset first
# For example, you could use one of seaborn's built-in datasets:
data = sns.load_dataset('EastWestAirlines')  # Using iris dataset as an example
# Or load your own data:
# data = pd.read_csv('your_data_file.csv')

# Pairplot to visualize relationships
sns.pairplot(data, diag_kind='kde')
plt.suptitle("Pairwise Feature Relationships", y=1.02)
plt.show()

# Correlation heatmap
plt.figure(figsize=(10,6))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title("Feature Correlation Heatmap")
plt.show()

ValueError: 'EastWestAirlines' is not one of the example datasets.

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt

# Pairplot to visualize relationships
sns.pairplot(data, diag_kind='kde')
plt.suptitle("Pairwise Feature Relationships", y=1.02)
plt.show()

# Correlation heatmap
plt.figure(figsize=(10,6))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title("Feature Correlation Heatmap")
plt.show()


ValueError: No variables found for grid columns.

#### Preprocessing

In [20]:
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import pandas as pd

# Standardize and cluster
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
linkage_matrix = linkage(scaled_data, method='ward')

# Plot dendrogram
plt.figure(figsize=(10, 5))
dendrogram(linkage_matrix)
plt.title("Hierarchical Clustering Dendrogram")
plt.show()

# Assign clusters (e.g., choose 3 clusters)
clusters = fcluster(linkage_matrix, t=3, criterion='maxclust')
data['HierCluster'] = clusters

# Cluster summary
cluster_summary = data.groupby('HierCluster').mean()
print(cluster_summary)


ValueError: could not convert string to float: 'flying patterns, earning and use of frequent flyer rewards, and use of the airline credit'

In [ ]:
# Exclude ID and Award? from features used for clustering
exclude_cols = ['ID#','Award?']
features = [c for c in num_cols if c not in exclude_cols]
print("Features used for clustering:", features)

In [ ]:
# Fill missing numeric (if any)
for c in features:
    if data[c].isnull().sum() > 0:
        data[c] = data[c].fillna(data[c].median())


In [ ]:
# IQR capping
data_feat = data[features].copy()
for c in data_feat.columns:
    q1 = data_feat[c].quantile(0.25)
    q3 = data_feat[c].quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5*iqr
    high = q3 + 1.5*iqr
    data_feat[c] = np.where(data_feat[c] < low, low, data_feat[c])
    data_feat[c] = np.where(data_feat[c] > high, high, data_feat[c])

In [ ]:
# Import the necessary library first
from sklearn.preprocessing import StandardScaler

# Make sure data_feat contains data
# For example, if data_feat is not defined or is empty:
import pandas as pd
import numpy as np

# Example: Create sample data if data_feat is not defined
# Replace this with your actual data source
data_feat = pd.DataFrame({
    'feature1': np.random.rand(10),
    'feature2': np.random.rand(10)
})

# Now apply scaling
scaler = StandardScaler()
X = scaler.fit_transform(data_feat)

In [ ]:
# PCA (2D) for visualization later
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X)


In [ ]:
#  K-Means experiments
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Assuming X is your data
# Let's determine the maximum number of clusters we can use
max_clusters = min(10, len(X) - 1)  # Ensure we don't exceed n_samples - 1
ks = list(range(2, max_clusters + 1))  # Start from 2 clusters
inertias = []
sil_scores = []
kmeans_models = {}

for k in ks:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X)
    
    # Check if each cluster has at least 2 samples
    unique_labels, counts = np.unique(labels, return_counts=True)
    if np.all(counts >= 2):  # Only calculate silhouette if all clusters have at least 2 samples
        inertias.append(km.inertia_)
        sil = silhouette_score(X, labels)
        sil_scores.append(sil)
        kmeans_models[k] = (km, labels)
        print(f"k={k} -> silhouette={sil:.4f}")
    else:
        print(f"k={k} -> skipped (some clusters have only one sample)")

In [ ]:
print('''Insights:

Clusters 0 and 2 are the most valuable groups for targeted promotions and loyalty benefits.

Cluster 1 indicates potential customers to re-engage via offers or discounts.

Cluster 3 represents steady, long-term members who may respond well to retention campaigns.''')

In [ ]:
# Plot Elbow
ks=ks[:2]
plt.figure()
plt.plot(ks, inertias, marker='o')
plt.title("Elbow curve (KMeans)")
plt.xlabel("k"); plt.ylabel("Inertia")
plt.show()

In [ ]:
# Plot silhouette vs k
plt.figure()
plt.plot(ks, sil_scores, marker='o')
plt.title("Silhouette scores (KMeans)")
plt.xlabel("k"); plt.ylabel("Silhouette")
plt.show()

In [ ]:
# Choose best k by silhouette
best_k = ks[np.argmax(sil_scores)]
best_sil = max(sil_scores)
print(f"Selected best_k = {best_k} with silhouette = {best_sil:.4f}")

best_km, best_labels = kmeans_models[best_k]


In [ ]:
# Visualize KMeans clusters on PCA 2D
plt.figure()
plt.scatter(X_pca[:,0], X_pca[:,1], c=best_labels)
plt.title(f"KMeans clusters (k={best_k}) on PCA (2D)")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()


#####  Hierarchical clustering

In [ ]:
Z = linkage(X, method='ward')
plt.figure(figsize=(12,5))
# truncated dendrogram for readability
dendrogram(Z, truncate_mode='level', p=5)
plt.title("Hierarchical dendrogram (truncated)")
plt.show()

In [ ]:
# Create a 4-cluster cut for interpretation
hc = AgglomerativeClustering(n_clusters=4, linkage='ward')
hc_labels = hc.fit_predict(X)

plt.figure()
plt.scatter(X_pca[:,0], X_pca[:,1], c=hc_labels)
plt.title("Hierarchical clustering (4 clusters) on PCA")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()

##### DBSCAN

In [ ]:
eps_list = np.linspace(0.3, 2.0, 10)   # scaled-space eps candidates
min_samples_list = [3,4,5,6,8]
best_db = None
best_db_score = -1
best_db_params = None
results = []
for eps in eps_list:
    for ms in min_samples_list:
        db = DBSCAN(eps=eps, min_samples=ms)
        labels = db.fit_predict(X)
        unique_labels = set(labels)
        n_clusters = len([lab for lab in unique_labels if lab != -1])
        if n_clusters >= 2:
            try:
                score = silhouette_score(X, labels)
            except:
                score = -1
        else:
            score = -1
        results.append({'eps': eps, 'min_samples': ms, 'n_clusters': n_clusters, 'silhouette': score})
        if score > best_db_score:
            best_db_score = score
            best_db = labels.copy()
            best_db_params = (eps, ms)

res_df = pd.DataFrame(results).sort_values(by='silhouette', ascending=False)
print("Top DBSCAN configurations by silhouette:")
print(res_df.head(10))
if best_db_params:
    print("Best DBSCAN:", best_db_params, "silhouette:", best_db_score)

    # visualize
    plt.figure()
    plt.scatter(X_pca[:,0], X_pca[:,1], c=best_db)
    plt.title(f"DBSCAN best (eps={best_db_params[0]:.3f}, min_samples={best_db_params[1]})")
    plt.show()
else:
    print("No DBSCAN configuration found with >=2 clusters in the grid tried.")


In [ ]:
# Append cluster assignments to original data and save
data_out = data.copy().reset_index(drop=True)
data_out['KMeans_cluster'] = len(best_labels)
data_out['Hierarchical_cluster_4'] = len(hc_labels)
data_out['DBSCAN_cluster'] = len(best_db) if best_db is not None else np.nan

out_csv = "EastWestAirlines_clusters.csv"
data_out.to_csv(out_csv, index=False)
print("Saved clustering output to:", out_csv)

In [ ]:
#Basic cluster profiling (for KMeans best_k)

profile = data_out.groupby('KMeans_cluster')[features].mean().transpose()
print("\nCluster profile (feature means per KMeans cluster):")
print(profile)

# Save profile summary
profile.to_csv("KMeans_cluster_profile.csv")

In [ ]:
from sklearn.metrics import silhouette_score

# --- K-Means Silhouette ---
kmeans_silhouette = silhouette_score(X, best_labels)
print(f"K-Means Silhouette Score: {kmeans_silhouette:.4f}")

# --- DBSCAN Silhouette ---
# Only compute if DBSCAN produced 2 or more clusters
if best_db is not None:
    unique_labels = set(best_db)
    n_clusters = len([lab for lab in unique_labels if lab != -1])
    if n_clusters >= 2:
        dbscan_silhouette = silhouette_score(X, best_db)
        print(f"DBSCAN Silhouette Score: {dbscan_silhouette:.4f}")
    else:
        print("DBSCAN did not produce enough clusters for a valid silhouette score.")
else:
    print("DBSCAN not executed or produced only one cluster.")
